# Data Preparation

Data found in the data set [18,393 Pitchfork articles on Kaggle.com](https://www.kaggle.com/datasets/nolanbconaway/pitchfork-data) can be converted to JSON line files using the code below.

Note that it requires the library `sqlite3` which is not included in the current package dependency definitions. You will need to install this library in order to run this code.

In [12]:
import sqlite3
import json
import os

documents_path = '../../05_src/documents/'
sqlite_file = os.path.join(documents_path, 'database.sqlite')


In [8]:
def sanitize_string(s):
    if isinstance(s, str):
        s = s.encode('utf-8', errors='ignore').decode('utf-8', errors='ignore')
        s = s.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
        s = s.replace('\u0720', ' ')
        s = s.replace("\n", " ")
    return s

In [9]:
def get_data_from_sqlite(db_path, table):
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute(f"SELECT * FROM {table}")
        rows = cursor.fetchall()
        columns = [description[0] for description in cursor.description]
        data = [dict(zip(columns, row)) for row in rows]
        data = [ {k: sanitize_string(v) for k, v in record.items()} for record in data ]
    return data

In [10]:
def create_jsonl_from_table(db_path, table):
    data = get_data_from_sqlite(db_path, table)
    output_file = os.path.join(documents_path, f'pitchfork_{table}.jsonl') 
    with open(output_file, 'w') as f:
        for record in data:    
            json_line = json.dumps(record)
            f.write(json_line + '\n')

In [13]:
table_list = ['artists', 'content', 'genres', 'labels', 'reviews', 'years']

for table in table_list:
    create_jsonl_from_table(sqlite_file, table)